# Are our significant DMGs the core enrichment genes in our identified enriched pathways?
We want to combine what we've found for statistically signficant differentially methylated genes and connect it to our identified enriched pathways so we can start to make sense of things biologically.

I have generated two csv files for both **phase 1 warm vs. control oysters**:
- phase1_wc_genes.csv - list of significant (adjusted p-value < 0.05) DMGs
- p1_wc_pathway.csv - list of enriched pathways from KEGG

Each enriched pathway contains a list of 'core enrichment genes' - these are a list of genes that are reported as part of the 'core enrichment' and contribute to the observed enrichment score.

The thinking is that maybe some of our signficant DMGs are part of that core enrichment group, which could tell us that that pathway is especially important/biologically relevant.

#### I. Load packages

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


#### II. Load, clean, and prep both csv files

In [4]:
# load in csv file
pathway <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/enriched_pathways/pathways_p1wc.csv')

# clean headers and columns
pathway <- pathway[,-1]

# checking dimensions
dim(pathway) #121 pathways, 11 rows of info/meta data

head(pathway)

[1] 121  11

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,cvn00053,Ascorbate and aldarate metabolism,12,0.7513381,1.627009,0.00618654,0.2537983,0.2296218,2374,"tags=58%, list=18%, signal=48%",111124535/111103451/111124599/111127562/111112920/111115614/111103498
2,cvn00910,Nitrogen metabolism,10,0.7751841,1.611862,0.01095923,0.2537983,0.2296218,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592
3,cvn00511,Other glycan degradation,37,0.5917499,1.578985,0.01320066,0.2537983,0.2296218,2221,"tags=30%, list=16%, signal=25%",111106921/111106925/111106928/111119851/111119435/111120040/111113388/111119434/111106926/111119431/111106930
4,cvn00592,alpha-Linolenic acid metabolism,11,0.7377666,1.564501,0.01887756,0.2537983,0.2296218,1602,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908
5,cvn03250,Viral life cycle - HIV-1,28,0.6023539,1.533878,0.01509720,0.2537983,0.2296218,2628,"tags=46%, list=19%, signal=37%",111124701/111124696/111129825/111111579/111108190/111135084/111128997/111124977/111106750/111123417/111130886/111104027/111135329
6,cvn00380,Tryptophan metabolism,31,0.5838480,1.517557,0.01807548,0.2537983,0.2296218,2374,"tags=42%, list=18%, signal=35%",111127901/111100724/111103451/111134248/111133558/111112920/111115614/111121380/111125148/111109254/111130627/111108303/111103498


I want to know if all of my genes provided in geneList are in the core enrichment, or if the core enrichment only includes a subset of my total genes

- total genes in my geneList for KEGG - around 13,000 genes
- total enriched pathways - 121
- total number of significant DMGs - 189
- total number of unique genes in the core_enrichment column - **805** (from code below)

since this number (805) does not match my total genes in the data set, I know that not all genes are represented in the core_enrichment column, and not all are significant DMGs

In [5]:
# Split the strings in 'genes_column' into lists
gene_lists <- strsplit(pathway$core_enrichment, '/')

# Flatten the lists
all_genes <- unlist(gene_lists)

# Count the unique genes
unique_genes_count <- length(unique(all_genes))

print(paste("Number of unique genes in the entire DataFrame:", unique_genes_count))


[1] "Number of unique genes in the entire DataFrame: 799"


In [7]:
# load data frame
genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/significant_genes/p1wc_lfc25_genes.csv')

# KEGG uses entrez IDs, which are my ensembl IDs without the 'LOC' in front of them, so need to convert those
genes$X <- substr(genes$X, start = 4, stop = nchar(genes$X))

# only grabbing the columns I care about
genes <- select(genes, X, log2FoldChange, padj)

# renaming columns to make more sense
colnames(genes) <- c('gene', 'lfc', 'padj')

# only selecting significant genes
genes <- filter(genes, genes$padj < 0.05)

# checking dimensions
dim(genes) # 344 sig DMGs

head(genes)

[1] 344   3

,gene,lfc,padj
,<chr>,<dbl>,<dbl>
1,111117672,1.314266,0.005811498
2,111128103,1.644558,0.001509994
3,111137770,2.655903,0.002201055
4,111125333,1.461069,0.017320572
5,111111295,2.360240,0.001233208
6,111125391,2.205531,0.001875643


#### III. Are the signficant genes in the core enrichment of the pathways?
Taking our list of siginificant genes, and going line by line in our pathways to see if our sig. gene matches any of the core enrichment genes

In [8]:
# generated from ChatGPT

# Sample dataframes
df1 <- genes
df2 <- pathway

# Function to check if any gene in df1 matches genes in a row of df2
get_gene_matches <- function(row_df2, df1_genes) {
  genes2 <- unlist(strsplit(as.character(row_df2), "/"))
  count <- sum(genes2 %in% df1_genes)
  return(count)
}

# Iterate over each row of df2
matches_count <- sapply(df2$core_enrichment, get_gene_matches, df1_genes = df1$gene)

# Add the matches count to df2
df2$MatchesCount <- matches_count

# Sort df with highest match counts at the top
gene_pathway_match <- df2[order(-df2$MatchesCount),]
head(gene_pathway_match)


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesCount
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
9,cvn04814,Motor proteins,111,0.4825362,1.425442,0.007216891,0.2537983,0.2296218,1179,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,8
13,cvn04144,Endocytosis,132,0.4453777,1.329073,0.027355623,0.2546177,0.2303631,3185,"tags=33%, list=24%, signal=25%",111120187/111125099/111112319/111119513/111112439/111119512/111136896/111104852/111102907/111107174/111136866/111125956/111134954/111112863/111116971/111112700/111135084/111133388/111134171/111121253/111115795/111127289/111129312/111134242/111121335/111121437/111129503/111135594/111106223/111123210/111125223/111105462/111104835/111104585/111104028/111133563/111105923/111119177/111101822/111104196/111137900/111112119/111123772,8
8,cvn00071,Fatty acid degradation,38,0.5657277,1.510768,0.017560053,0.2537983,0.2296218,3018,"tags=42%, list=22%, signal=33%",111127947/111113990/111128659/111115744/111107112/111115745/111103451/111112920/111115614/111135553/111121380/111117093/111103498/111129333/111107779/111134458,5
45,cvn04148,Efferocytosis,98,0.4000467,1.167589,0.194501018,0.6325836,0.5723244,1406,"tags=14%, list=10%, signal=13%",111135946/111113124/111121723/111134726/111117970/111110712/111134431/111128659/111117731/111134947/111130916/111127576/111130408/111112330,5
46,cvn03040,Spliceosome,92,0.3988463,1.153711,0.216049383,0.6325836,0.5723244,1557,"tags=16%, list=11%, signal=15%",111112733/111137770/111119513/111119512/111129112/111133954/111121854/111121021/111118318/111119442/111134531/111114893/111136440/111135640/111136164,5
4,cvn00592,alpha-Linolenic acid metabolism,11,0.7377666,1.564501,0.018877557,0.2537983,0.2296218,1602,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908,4


above returns just the ***number*** of significant genes in the core enrichment for each pathway

below returns the ***percent*** of significant genes in the core enrichment

it's interesting that the percent returns a much different top 5 ... maybe this is the better way to do it?

In [11]:
# Sample dataframes
df1 <- genes
df2 <- pathway

# Function to check if any gene in df1 matches genes in a row of df2
get_gene_matches <- function(row_df2, df1_genes) {
  genes2 <- unlist(strsplit(as.character(row_df2), "/"))
  count <- sum(genes2 %in% df1_genes)
  # Calculate the percentage of matched genes in the row
  percentage <- count / length(genes2) * 100
  return(percentage)
}

# Iterate over each row of df2
matches_percentage <- sapply(df2$core_enrichment, get_gene_matches, df1_genes = df1$gene)

# Add the matches percentage to df2
df2$MatchesPercentage <- matches_percentage

# Sort df with highest match percentages at the top
gene_pathway_match <- df2[order(-df2$MatchesPercentage),]
head(gene_pathway_match, 10)


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesPercentage
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
4,cvn00592,alpha-Linolenic acid metabolism,11,0.7377666,1.5645014,0.018877557,0.2537983,0.2296218,1602,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908,80.00000
15,cvn01040,Biosynthesis of unsaturated fatty acids,18,0.6354291,1.4894918,0.033088235,0.2669118,0.2414861,1602,"tags=33%, list=12%, signal=29%",111113990/111115744/111107112/111115745/111119293/111124908,66.66667
2,cvn00910,Nitrogen metabolism,10,0.7751841,1.6118619,0.010959229,0.2537983,0.2296218,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,60.00000
32,cvn04512,ECM-receptor interaction,23,0.5113610,1.2737730,0.176331361,0.6096027,0.5515327,1679,"tags=26%, list=12%, signal=23%",111114114/111111291/111113265/111134722/111109289/111119175,50.00000
71,cvn03083,Polycomb repressive complex,38,0.3891222,1.0391458,0.438264739,0.7377131,0.6674394,865,"tags=11%, list=6%, signal=10%",111123912/111104330/111104315/111123056,50.00000
87,cvn00532,Glycosaminoglycan biosynthesis - chondroitin sulfate / dermatan sulfate,12,0.4314562,0.9343104,0.569148936,0.7915750,0.7161705,1400,"tags=17%, list=10%, signal=15%",111118293/111111504,50.00000
94,cvn03430,Mismatch repair,21,0.3553110,0.8641842,0.647272727,0.8331915,0.7538227,330,"tags=10%, list=2%, signal=9%",111121171/111137687,50.00000
9,cvn04814,Motor proteins,111,0.4825362,1.4254420,0.007216891,0.2537983,0.2296218,1179,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,40.00000
45,cvn04148,Efferocytosis,98,0.4000467,1.1675889,0.194501018,0.6325836,0.5723244,1406,"tags=14%, list=10%, signal=13%",111135946/111113124/111121723/111134726/111117970/111110712/111134431/111128659/111117731/111134947/111130916/111127576/111130408/111112330,35.71429


now that I have a df with counts of number of matches between core enrichment genes and significant DMGs, want to only look at those with matches (filter out any pathways that did not contain sig. DMGs in their core enrichment)

In [14]:
# only want to look at pathways that have significant genes in their core enrichment
matched_pathways <- filter(gene_pathway_match, gene_pathway_match$MatchesPercentage != 0)

# checking dimensions to see how many pathways we have 
dim(matched_pathways) # 77 matches

# looking at df
head(matched_pathways)

[1] 77 12

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesPercentage
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
1,cvn00592,alpha-Linolenic acid metabolism,11,0.7377666,1.5645014,0.01887756,0.2537983,0.2296218,1602,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908,80.00000
2,cvn01040,Biosynthesis of unsaturated fatty acids,18,0.6354291,1.4894918,0.03308824,0.2669118,0.2414861,1602,"tags=33%, list=12%, signal=29%",111113990/111115744/111107112/111115745/111119293/111124908,66.66667
3,cvn00910,Nitrogen metabolism,10,0.7751841,1.6118619,0.01095923,0.2537983,0.2296218,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,60.00000
4,cvn04512,ECM-receptor interaction,23,0.5113610,1.2737730,0.17633136,0.6096027,0.5515327,1679,"tags=26%, list=12%, signal=23%",111114114/111111291/111113265/111134722/111109289/111119175,50.00000
5,cvn03083,Polycomb repressive complex,38,0.3891222,1.0391458,0.43826474,0.7377131,0.6674394,865,"tags=11%, list=6%, signal=10%",111123912/111104330/111104315/111123056,50.00000
6,cvn00532,Glycosaminoglycan biosynthesis - chondroitin sulfate / dermatan sulfate,12,0.4314562,0.9343104,0.56914894,0.7915750,0.7161705,1400,"tags=17%, list=10%, signal=15%",111118293/111111504,50.00000


In [11]:
write.csv(matched_pathways, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_kegg_pathways/p1wc_siggene_pathways.csv')

In [17]:
mean(matched_pathways$MatchesPercentage)
median(matched_pathways$MatchesPercentage)
sd(matched_pathways$MatchesPercentage)

[1] 21.27795

[1] 18.18182

[1] 14.76078

**Stats on Matched Percentage**

25 pathways with only 1 match

267 pathways with 0 matches

- mean % of matches: 21%
- median % of matches: 18%
- standard deviation of matches: 14%

**alpha-Linolenic acid metabolism** have the highest percent of significant DMGs in its core enrichment with 80%

## Investigating adjusted p-values with NA values
We're trying to figure out *why* we get NA values so we can decide if we want to exclude or keep those genes in our pathway analysis

Starting with looking at our list of genes with stat info from DESeq (normal lfcShrink and lfcThreshold=0.5)

In [ ]:
unfilter_genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_sig_genes/unfiltered_phase1_wc_genes.csv')
head(unfilter_genes)

filtering df to only include genes with NA for adjusted p-value

In [ ]:
genes_w_na <- unfilter_genes[is.na(unfilter_genes$padj),]
dim(genes_w_na) # 5,776 genes with NA for adjusted p-value

looking more into the stats of those genes with NA for adjusted p-value, specifically looking at the baseMean (since this is what the DESeq documentation points at)
> baseMean - the average of the normalized count values, dividing by size factors, taken over all samples

In [ ]:
mean(genes_w_na$baseMean) # average = 4.5 counts per sample
sd(genes_w_na$baseMean) # standard deviation = 22

loading in counts matrix that was generated with featureCounts to pull out the genes with NA for adjusted p-value

In [ ]:
# loading in counts matrix
counts_matrix <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/featureCounts_meta.csv')
head(counts_matrix)

In [ ]:
# setting genees as row names
rownames(counts_matrix) = counts_matrix$X

# removing gene column (since now are rownames)
counts_matrix2 <- counts_matrix[,-1]
head(counts_matrix2)

In [ ]:
# replace the '.' with '-'
cleaned_column_names2 <- gsub('\\.', "-", colnames(counts_matrix2))
head(cleaned_column_names2)

# now assigning to the columns
colnames(counts_matrix2) = cleaned_column_names2
head(counts_matrix2)

In [ ]:
# creating new df of counts matrix of genes with NA for padj
counts_na <- counts_matrix2[rownames(counts_matrix2) %in% genes_w_na$X,]
dim(counts_na) # 5,776 genes 
head(counts_na)

also need to load in meta data so that I can pull out the right columns aka samples that were warm or control for phase 1

In [ ]:
meta <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/CV_CE18_meta.csv')
head(meta)

pulling out only the samples that were either warm or control for phase 1 treatment (we're ignoring the effects of phase 2 for this analysis)

In [ ]:
p1_wc_meta <- filter(meta, meta$Phase1 == 'warm' | meta$Phase1 == 'control')
dim(p1_wc_meta) # 15 total samples
head(p1_wc_meta)

In [ ]:
p1_wc_meta

In [ ]:
# only looking at samples with control or warm for phase 1 in the counts matrix
filtered_df <- counts_na[,colnames(counts_na) %in% p1_wc_meta$unique_ID]
dim(filtered_df) # still seeing 5,776 genes with NA for padj and info for 15 samples
head(filtered_df,20)

all of the **W##** samples are **control** and all of the **G##** samples are **warm** for phase 1

looking at this df in excel to more easily look at why we might get NA for those genes

In [ ]:
write.csv(filtered_df, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/featureCounts_analysis/na_p1_wc_countsmatrix.csv')

# Unfiltered DESeq DF and Pathway
still looking at significantly differentially methylated genes for phase 1 warm vs. control, but now genes get assigned NA for adjusted p-values only when the gene has low counts (so this now includes outliers, instead of assigning outliers NA), therefore, different genes will be kept in the analysis for enriched pathways

In [ ]:
# load in csv file
pathway <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_kegg_pathways/unfiltered_pathways_p1wc.csv')

# clean headers and columns
pathway <- pathway[,-1]

# checking dimensions
dim(pathway) #119 pathways, 11 rows of info/meta data

head(pathway)

In [ ]:
# load data frame
genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/unfiltered_lfc_p1wc.csv')

# KEGG uses entrez IDs, which are my ensembl IDs without the 'LOC' in front of them, so need to convert those
genes$X <- substr(genes$X, start = 4, stop = nchar(genes$X))

# only grabbing the columns I care about
genes <- select(genes, X, log2FoldChange, padj)

# renaming columns to make more sense
colnames(genes) <- c('gene', 'lfc', 'padj')

# only selecting genes that have padj < 0.05
genes <- filter(genes, genes$padj < 0.05)

# checking dimensions
dim(genes) # 189 sig DMGs

head(genes)

#### III. Are the signficant genes in the core enrichment of the pathways?
Taking our list of siginificant genes, and going line by line in our pathways to see if our sig. gene matches any of the core enrichment genes

In [ ]:
# generated from ChatGPT

# Sample dataframes
df1 <- genes
df2 <- pathway

# Function to check if any gene in df1 matches genes in a row of df2
get_gene_matches <- function(row_df2, df1_genes) {
  genes2 <- unlist(strsplit(as.character(row_df2), "/"))
  count <- sum(genes2 %in% df1_genes)
  return(count)
}

# Iterate over each row of df2
matches_count <- sapply(df2$core_enrichment, get_gene_matches, df1_genes = df1$gene)

# Add the matches count to df2
df2$MatchesCount <- matches_count

# Sort df with highest match counts at the top
gene_pathway_match <- df2[order(-df2$MatchesCount),]
head(gene_pathway_match)


now that I have a df with counts of number of matches between core enrichment genes and significant DMGs, want to only look at those with matches (filter out any pathways that did not contain sig. DMGs in their core enrichment)

In [ ]:
# only want to look at pathways that have significant genes in their core enrichment
matched_pathways <- filter(gene_pathway_match, gene_pathway_match$MatchesCount != 0)

# checking dimensions to see how many pathways we have 
dim(matched_pathways) # 41 matches

# looking at df
head(matched_pathways)

In [ ]:
mean(matched_pathways$MatchesCount)
median(matched_pathways$MatchesCount)
sd(matched_pathways$MatchesCount)

**Stats on Matched Counts**

25 pathways with only 1 match

148 pathways with 0 matches

- mean number of matches: 1.8
- median number of matches: 1
- standard deviation of matches: 1.31

**motor proteins** have the highest number of significant DMGs in its core enrichment with 7 genes

**these results are the *same* as without filtering the outlier counts**

In [ ]:
unfilter_genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_sig_genes/unfiltered_phase1_wc_genes.csv')
head(unfilter_genes)

filtering df to only include genes with NA for adjusted p-value

In [ ]:
genes_w_na <- unfilter_genes[is.na(unfilter_genes$padj),]
dim(genes_w_na) # 5,776 genes with NA for adjusted p-value

looking more into the stats of those genes with NA for adjusted p-value, specifically looking at the baseMean (since this is what the DESeq documentation points at)
> baseMean - the average of the normalized count values, dividing by size factors, taken over all samples

In [ ]:
mean(genes_w_na$baseMean) # average = 4.5 counts per sample
sd(genes_w_na$baseMean) # standard deviation = 22

loading in counts matrix that was generated with featureCounts to pull out the genes with NA for adjusted p-value

In [ ]:
# loading in counts matrix
counts_matrix <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/featureCounts_meta.csv')
head(counts_matrix)

In [ ]:
# setting genees as row names
rownames(counts_matrix) = counts_matrix$X

# removing gene column (since now are rownames)
counts_matrix2 <- counts_matrix[,-1]
head(counts_matrix2)

In [ ]:
# replace the '.' with '-'
cleaned_column_names2 <- gsub('\\.', "-", colnames(counts_matrix2))
head(cleaned_column_names2)

# now assigning to the columns
colnames(counts_matrix2) = cleaned_column_names2
head(counts_matrix2)

In [ ]:
# creating new df of counts matrix of genes with NA for padj
counts_na <- counts_matrix2[rownames(counts_matrix2) %in% genes_w_na$X,]
dim(counts_na) # 5,776 genes 
head(counts_na)

also need to load in meta data so that I can pull out the right columns aka samples that were warm or control for phase 1

In [ ]:
meta <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/CV_CE18_meta.csv')
head(meta)

pulling out only the samples that were either warm or control for phase 1 treatment (we're ignoring the effects of phase 2 for this analysis)

In [ ]:
p1_wc_meta <- filter(meta, meta$Phase1 == 'warm' | meta$Phase1 == 'control')
dim(p1_wc_meta) # 15 total samples
head(p1_wc_meta)

In [ ]:
p1_wc_meta

In [ ]:
# only looking at samples with control or warm for phase 1 in the counts matrix
filtered_df <- counts_na[,colnames(counts_na) %in% p1_wc_meta$unique_ID]
dim(filtered_df) # still seeing 5,776 genes with NA for padj and info for 15 samples
head(filtered_df,20)

all of the **W##** samples are **control** and all of the **G##** samples are **warm** for phase 1

looking at this df in excel to more easily look at why we might get NA for those genes

In [ ]:
write.csv(filtered_df, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/featureCounts_analysis/na_p1_wc_countsmatrix.csv')